In [58]:
import pandas as pd
from pandas import json_normalize
import time 
from sklearn import preprocessing
import numpy as np

import tensorflow as tf

def getWeather(lat, long, dt):
   print("Gathering Weather Data")
   LAT = lat
   LONG = long
   dt = dt
   API_KEY = '2fc271ed0e74cba6a60d713a61a9770b'
   url = "http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=%s&lon=%s&dt=%s&units=imperial&appid=%s" % (LAT, LONG, dt, API_KEY)
   print(url)
   response = requests.get(url)
   data = json.loads(response.text)
   #print(data)

   return data

def format(data):
   print("Formating Weather Data")
   hourly = data["hourly"]
   for entry in hourly:
      entry.pop('feels_like')
      entry.pop('pressure')
      entry.pop('dew_point')
      entry.pop('uvi')
      entry.pop('clouds')
      entry.pop('visibility')
      entry.pop('wind_deg')
      entry.pop('weather')


   df = json_normalize(hourly)
   df.to_csv('data.csv')
   return df
   print("file saved")
   #print(df)
    
def generateReport(coords):
   lon, lat = coords
   today = int(time.time())
   print("time" + str(today))
   yesterday = today - 86400
   print("first call")
   result = getWeather(lon, lat, str(yesterday))
   #print('second call')
   #result =  {**getWeather(lon, lat, str(today)), **getWeather(lon, lat, str(yesterday))}
   #print(result)
   format(result)
    
    

   

In [51]:
LAT = "-75"
LON = "40"
coords = [LON, LAT]
generateReport(coords)


time1636830917
first call
Gathering Weather Data
http://api.openweathermap.org/data/2.5/onecall/timemachine?lat=40&lon=-75&dt=1636744517&units=imperial&appid=2fc271ed0e74cba6a60d713a61a9770b
Formating Weather Data


In [56]:
temp = pd.read_csv("data.csv")

temp.drop('Unnamed: 0', inplace=True, axis=1)
temp.drop('wind_gust', inplace=True, axis=1)
temp.drop('rain.1h', inplace=True, axis=1)
temp['dt'] = pd.to_datetime(temp['dt'],unit='s')
temp.index =  temp['dt']
temp.drop('dt', inplace=True, axis=1)
temp_avg = temp["temp"].mean()
temp_std = temp['temp'].std()
temp

,temp,humidity,wind_speed
dt,,,
2021-11-12 00:00:00,56.89,82,4.00
2021-11-12 01:00:00,56.52,85,4.00
2021-11-12 02:00:00,56.48,89,1.99
2021-11-12 03:00:00,55.99,91,3.44
2021-11-12 04:00:00,56.32,93,1.01
2021-11-12 05:00:00,58.10,92,1.01
2021-11-12 06:00:00,59.58,89,1.01
2021-11-12 07:00:00,61.75,88,1.99
2021-11-12 08:00:00,61.21,89,1.01


In [53]:
model = tf.keras.models.load_model("weather.model")

In [59]:
norm = temp
#norm.drop('DATE', inplace=True, axis=1)
x = norm.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
temp = pd.DataFrame(x_scaled)
genPred = temp.to_numpy()
genPred = genPred[: 24, 0]
genPred = np.reshape(genPred, (1, genPred.shape[0], 1))
model.predict(genPred)
pred = [(genPred * temp_std) + temp_avg]
return pred

[array([[[61.21257295],
         [61.11364895],
         [61.10295447],
         [60.97194701],
         [61.06017652],
         [61.53608117],
         [61.93177717],
         [62.51195306],
         [62.36757749],
         [62.50393219],
         [62.78198884],
         [63.05202463],
         [63.56268636],
         [63.13758052],
         [63.1723376 ],
         [62.66167587],
         [62.39164009],
         [62.10021533],
         [62.3060842 ],
         [62.81674592],
         [62.96646873],
         [62.31677868],
         [61.20989933],
         [60.31958333]]])]

AttributeError: 'list' object has no attribute 'info'